# 3.2 Comparison with other programming frameworks

In [1]:
import pennylane as qml

from pennylane import numpy as pnp
import numpy as np

import qiskit
from qiskit.compiler import transpile
from qiskit.transpiler.basepasses import TransformationPass
from qiskit.transpiler import PassManager
from qiskit.circuit import QuantumRegister, QuantumCircuit, Parameter
from qiskit.opflow import Z,StateFn, CircuitStateFn
from qiskit.opflow.gradients import Gradient

In the examples below, we use Qiskit to reproduce some of the transforms functionality of PennyLane.  

## Example 1: overrotation with a differentiable parameter

We define below a `TransformationPass` that overrotates all `RX` gates by a specified angle.

In [2]:
class OverrotateRX(TransformationPass):
    def __init__(self, angle):
        self.angle = angle
        super().__init__()
        
    def run(self, dag):
        for node in dag.op_nodes():
            if node.name == 'rx':
                node.op.params[0] += self.angle
        return dag

We test this on a simple subroutine that itself takes a parameter in its `RX` gate.

In [3]:
def subroutine(theta):
    q = QuantumRegister(1, 'q')
    circ = QuantumCircuit(q)
    circ.rx(theta, q[0])
    circ.h(q[0])
    circ.ry(0.1, q[0])
    return circ

Below, we compute the gradient of both the subroutine parameters, and the parameter of the `TransformationPass`.

In [4]:
def evaluate_grad_with_qiskit(params, t_params):
    p0, p1, t0, t1 = Parameter('p0'), Parameter('p1'), Parameter('t0'), Parameter('t1')
    
    # We create a circuit that applies the subroutine twice, each time
    # with a different parameter for the transform.
    circ = PassManager(OverrotateRX(t0)).run(subroutine(p0))
    circ.compose(PassManager(OverrotateRX(t1)).run(subroutine(p1)), inplace=True)
    
    # Now let's compute the gradients w.r.t. all of the parameters
    op = ~StateFn(Z) @ CircuitStateFn(primitive=circ, coeff=1.)
    
    # Use Qiskit's built-in parameter-shift gradient computation
    grad = Gradient(grad_method='param_shift').convert(operator=op, params=[p0, p1, t0, t1])
    
    return grad.assign_parameters({p0: params[0], p1: params[1], t0: t_params[0], t1: t_params[1]}).eval()


params = np.array([0.3, 0.4])
t_params = np.array([0.1, 0.2])
print(evaluate_grad_with_qiskit(params, t_params))

[(-0.4406608121346723+0j), (-0.03726993167698134+0j), (-0.4406608121346723+0j), (-0.03726993167698134+0j)]


Let's do the same computation in PennyLane using a `qfunc_transform`. Note that to get the gradient, we need only add the `@qml.gradients.param_shift` decorator. The results of the gradient computation are also tensors which can be fed forward into other differentiable computations.

In [5]:
dev = qml.device('default.qubit', wires=1)

@qml.qfunc_transform
def overrotate_rx(tape, epsilon):
    for op in tape:
        if op.name == 'RX':
            qml.RX(op.data[0] + epsilon, wires=op.wires[0])
        else:
            qml.apply(op)

def subroutine(theta):
    qml.RX(theta, wires=0)
    qml.Hadamard(wires=0)
    qml.RY(0.1, wires=0)

@qml.gradients.param_shift
@qml.qnode(dev)
def evaluate_grad_with_pl(params, transform_params):
    overrotate_rx(transform_params[0])(subroutine)(params[0])
    overrotate_rx(transform_params[1])(subroutine)(params[1])
    return qml.expval(qml.PauliZ(0))

params = pnp.array([0.3, 0.4], requires_grad=True)
t_params = pnp.array([0.1, 0.2], requires_grad=True)
print(evaluate_grad_with_pl(params, t_params))

(tensor([-0.44066081, -0.03726993], requires_grad=True), tensor([-0.44066081, -0.03726993], requires_grad=True))


## Example 2: autodifferentiation and transpilation compilation pass 

In the example below, we will transpile a circuit and compute gradients.

In [6]:
p0, p1 = Parameter('p0'), Parameter('p1')
    
# A simple circuit with two RX that should get merged.
q = QuantumRegister(1, 'q')
circ = QuantumCircuit(q)
circ.rx(p0, q[0])
circ.rx(p1, q[0])

In [7]:
circ.draw()

┌────────┐┌────────┐
q: ┤ Rx(p0) ├┤ Rx(p1) ├
   └────────┘└────────┘

In [8]:
transpiled_circ = transpile(circ, optimization_level=2)
transpiled_circ.draw()

┌────────┐┌────────┐
q: ┤ Rx(p0) ├┤ Rx(p1) ├
   └────────┘└────────┘

The transpiler doesn't operate on parametrized circuits, so let's assign some values and then transpile.

In [9]:
params = np.array([0.3, 0.4])
circ_with_params = circ.assign_parameters({p0: params[0], p1: params[1]})

In [10]:
circ_with_params.draw()

┌─────────┐┌─────────┐
q: ┤ Rx(0.3) ├┤ Rx(0.4) ├
   └─────────┘└─────────┘

In [11]:
transpiled_circ_with_params = transpile(circ_with_params, optimization_level=2)
transpiled_circ_with_params.draw()

┌──────────────────┐
q: ┤ U3(0.7,-π/2,π/2) ├
   └──────────────────┘

Now let's compute the gradients with respect to the two parameters; we get a total of 4 (two shifts per parameter). Note that the output is structured in a way that is similar to a batch transform: there is a list of operations, and coefficients detailing how execution results should be multiplied and summed together.

In [12]:
op = ~StateFn(Z) @ CircuitStateFn(primitive=transpiled_circ, coeff=1.)

grad = Gradient(grad_method='param_shift').convert(operator=op, params=[p0, p1])
grad_circuits = grad.assign_parameters({p0: params[0], p1: params[1]})

print(grad_circuits)

ListOp([
  SummedOp([
    0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
         ┌─────────────────────┐┌─────────┐
      q: ┤ Rx(1.8707963267949) ├┤ Rx(0.4) ├
         └─────────────────────┘└─────────┘
      )
    ]),
    -0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
         ┌──────────────────────┐┌─────────┐
      q: ┤ Rx(-1.2707963267949) ├┤ Rx(0.4) ├
         └──────────────────────┘└─────────┘
      )
    ])
  ]),
  SummedOp([
    0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
         ┌─────────┐┌─────────────────────┐
      q: ┤ Rx(0.3) ├┤ Rx(1.9707963267949) ├
         └─────────┘└─────────────────────┘
      )
    ]),
    -0.5 * ComposedOp([
      OperatorMeasurement(Z),
      CircuitStateFn(
         ┌─────────┐┌──────────────────────┐
      q: ┤ Rx(0.3) ├┤ Rx(-1.1707963267949) ├
         └─────────┘└──────────────────────┘
      )
    ])
  ])
])


Let's actually evaluate the gradient now:

In [13]:
res = grad.assign_parameters({p0: params[0], p1: params[1]}).eval()
print(res)

[(-0.6442176872376912+0j), (-0.644217687237691+0j)]


Note that transpilation is not performed in the above, even though we have parameter values assigned, and use the transpiled circuits. 

Instead, we can apply a transpilation pass to the constructed gradient circuits directly:

In [14]:
extracted_grad_circuits = [
    grad_circuits[0][0][1].to_circuit(),
    grad_circuits[0][1][1].to_circuit(), 
    grad_circuits[0][0][1].to_circuit(),
    grad_circuits[0][1][1].to_circuit()
]

transpiled_grad_circuits = transpile(extracted_grad_circuits, optimization_level=2)

In [15]:
for t_grad_circuit in transpiled_grad_circuits:
    print(t_grad_circuit.draw())

   ┌─────────────────────┐
q: ┤ U3(2.2708,-π/2,π/2) ├
   └─────────────────────┘
   ┌─────────────────────┐
q: ┤ U3(0.8708,π/2,-π/2) ├
   └─────────────────────┘
   ┌─────────────────────┐
q: ┤ U3(2.2708,-π/2,π/2) ├
   └─────────────────────┘
   ┌─────────────────────┐
q: ┤ U3(0.8708,π/2,-π/2) ├
   └─────────────────────┘


Even though they are optimized, we would still be executing four circuits. Now, recall that we had previously transpiled the circuit and assigned parameter values. Presumably with just one gate, we could compute the gradient with two evaluations. However, once the parameters are assigned there are no more differentiable parameters and so no gradient can be computed.

In [16]:
transpiled_circ_with_params = transpile(circ_with_params, optimization_level=2)
transpiled_circ_with_params.draw()

┌──────────────────┐
q: ┤ U3(0.7,-π/2,π/2) ├
   └──────────────────┘

In [17]:
op = ~StateFn(Z) @ CircuitStateFn(primitive=transpiled_circ_with_params, coeff=1.)

grad = Gradient(grad_method='param_shift').convert(operator=op, params=[p0, p1])

ValueError: The operator we are taking the gradient of is not parameterized!

Now we do the same computation in PennyLane. We can stack a couple transforms here; first we merge the rotation gates, create a QNode, and take the gradient using the parameter-shift rule. Note that in this structure, every time we run this QNode, the gradient is what is returned.

In [18]:
@qml.gradients.param_shift
@qml.qnode(dev)
@qml.transforms.merge_rotations()
def evaluate_grad_with_pl(params):
    qml.RX(params[0], wires=0)
    qml.RX(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

In [19]:
params = pnp.array([0.3, 0.4], requires_grad=True)

In [20]:
evaluate_grad_with_pl(params)

tensor([-0.64421769, -0.64421769], requires_grad=True)

The output is again a tensor that is still differentiable. Furthermore, we can see below that only two circuits are executed for the gradient. After merging the two gates, the parameter-shift rule was applied to the joint parameter.

In [21]:
print(qml.draw(evaluate_grad_with_pl)(params))

0: ──RX(2.27)─┤  <Z>

0: ──RX(-0.87)─┤  <Z>
